In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
#there are some irregular lines in the data, so setting argument to get a note of which lines and skip them when reading in
accre_gpu = pd.read_csv('../data/accre-gpu-jobs-2022-v2.csv', on_bad_lines='warn')

In [ ]:
#per our stakeholder, no need to include jobs with state of pending or running in our analysis, so dropping those 1258 rows
accre_gpu = accre_gpu.loc[~accre_gpu['STATE'].isin(['PENDING', 'RUNNING'])]

#stripping the 'M' from USEDMEM column and converting it to a numeric data type
accre_gpu['USEDMEM'] = accre_gpu['USEDMEM'].str.strip(to_strip='M').astype(float)

#converting REQTIME and USEDTIME columns to timedelta data types
accre_gpu['REQTIME'] = pd.to_timedelta(accre_gpu['REQTIME'].str.replace('-',' days '))
accre_gpu['USEDTIME'] = pd.to_timedelta(accre_gpu['USEDTIME'].str.replace('-',' days '))

#adding in a column that gives the difference in the time requested and time used
accre_gpu['time_dif'] = accre_gpu['REQTIME'] - accre_gpu['USEDTIME']

#adding in columns that provide versions of the other time columns but in seconds
accre_gpu['REQTIME_s'] = accre_gpu['REQTIME'].astype('timedelta64[s]')
accre_gpu['USEDTIME_s'] = accre_gpu['USEDTIME'].astype('timedelta64[s]')
accre_gpu['time_dif_s'] = accre_gpu['REQTIME_s'] - accre_gpu['USEDTIME_s']

accre_gpu

Doing a little exploration of the values in our columns

In [ ]:
accre_gpu['STATE'].value_counts().sort_index(ascending=False)

After dropping jobs marked as pending or running, these are the STATE column values that remain:   
- Out of memory
- Failed
- Completed
- Cancelled
- Cancelled by [numerical code] (92 versions of this)

Exploring the jobs that show some version of a Cancelled state

In [ ]:
cancelled = accre_gpu.loc[accre_gpu['STATE'].str.contains('CANCELLED')]
cancelled

There are 51,916 jobs that have a STATE value of some version of Cancelled. May be worth looking at how much time and memory was spent on these jobs

In [ ]:
#taking a quick look just to see how many of the cancelled jobs ran for more than an hour
cancelled.loc[cancelled['USEDTIME'] > '01:00:00'].describe()

Only 295 Cancelled jobs ran for longer than an hour but the mean and median for these is around 2 days, so, again, may be worth exploring to better understand them and see if such instances can be avoided

In [ ]:
cancelled.loc[cancelled['USEDTIME'] > '01:00:00'].groupby('STATE')['EXITCODE'].value_counts()

Do these exit codes suggest that there tends to be an issue with the environment that is associated with the job being cancelled?

Shifting gears back to the primary dataset

In [ ]:
#per our stakeholder, CANCELLED jobs can be excluded from the main analysis, so dropping them from the primary dataframe
accre_gpu = accre_gpu.loc[~accre_gpu['STATE'].str.contains('CANCELLED')]
accre_gpu.info()

In [ ]:
accre_gpu['EXITCODE'].value_counts()

In [ ]:
#Taking a look at the exit codes associated with failed jobs
accre_gpu.loc[accre_gpu['STATE'] == 'FAILED']['EXITCODE'].value_counts()

Based on exit codes, the failed jobs seem to fail due to user side issues

In [ ]:
#looking at overall distribution of values for our numerical/time columns
accre_gpu.describe()

In [ ]:
#interestingly, values slightly lower if only looking at completed jobs (no failed(440) or out of memory(3))
accre_gpu.loc[accre_gpu['STATE'] == 'COMPLETED'].describe()

So much skew!

In [ ]:
#looking at the longest running jobs
accre_gpu.nlargest(20, 'USEDTIME')

In [ ]:
#looking at the shortest running jobs
accre_gpu.loc[accre_gpu['EXITCODE'] == '0:0'].nsmallest(20, 'USEDTIME')

Interesting that the very longest failed after running for 49 days, but used no memory?   

Also, for all these jobs that ran for just over 5 days, did they really complete within a few minutes of hitting that 5 day mark, or were they forced to "complete" because they hit the requested time amount?

In [ ]:
accre_gpu.loc[accre_gpu['USEDTIME'] > '24:00:00']

Approximately 2.8% of jobs that show a status of completed, failed, or out of memory run for longer than 12 hours  
Approximately 1.6% run longer than 24 hours

In [ ]:
#looking at the stats for jobs that failed or ran out of memory
accre_gpu.loc[accre_gpu['STATE'] != 'COMPLETED'].describe()

In [ ]:
#total number of failed/out of memory jobs
accre_gpu.loc[accre_gpu['STATE'] != 'COMPLETED'].groupby('PARTITION')['JOBID'].count()

In [ ]:
#total amount of time spent on failed/out of memory jobs
accre_gpu.loc[accre_gpu['STATE'] != 'COMPLETED'].groupby('PARTITION')['USEDTIME'].sum()

In [ ]:
#total amount of memory used on failed/out of memory jobs
accre_gpu.loc[accre_gpu['STATE'] != 'COMPLETED'].groupby('PARTITION')['USEDMEM'].sum()

In [ ]:
#total number of GPUs used on failed/out of memory jobs
accre_gpu.loc[accre_gpu['STATE'] != 'COMPLETED'].groupby('PARTITION')['GPUS'].sum()

Looking at general stats for completed/failed/out of memory jobs for each partition

In [ ]:
accre_gpu.loc[accre_gpu['PARTITION'] == 'maxwell']['EXITCODE'].value_counts()

In [ ]:
accre_gpu.loc[(accre_gpu['PARTITION'] == 'maxwell')
              &
              (accre_gpu['EXITCODE'] == '0:0')].describe()

In [ ]:
accre_gpu.loc[(accre_gpu['PARTITION'] == 'pascal')
              &
              (accre_gpu['EXITCODE'] == '0:0')].describe()

In [ ]:
accre_gpu.loc[(accre_gpu['PARTITION'] == 'turing')
              &
              (accre_gpu['EXITCODE'] == '0:0')].describe()

In [ ]:
#plotting USEDTIME(in minutes) against USEDMEM (dropping that 1 outlier that failed after 49 days)
sns.set(rc={"figure.figsize":(12, 8)})
g = sns.FacetGrid(accre_gpu.loc[accre_gpu['USEDTIME']<'49days'], col='PARTITION', hue='PARTITION')
g.map_dataframe(sns.scatterplot, 
      x='USEDTIME_s', 
      y="USEDMEM");

In [ ]:
#plotting GPUs against USEDMEM (dropping that 1 outlier that failed after 49 days)
sns.set(rc={"figure.figsize":(12, 8)})
g = sns.FacetGrid(accre_gpu.loc[accre_gpu['USEDTIME']<'49days'], col='PARTITION', hue='PARTITION')
g.map_dataframe(sns.scatterplot, 
      x='GPUS', 
      y="USEDMEM");

In [ ]:
#plotting GPUs against USEDTIME(in minutes) (dropping that 1 outlier that failed after 49 days)
sns.set(rc={"figure.figsize":(12, 8)})
g = sns.FacetGrid(accre_gpu.loc[accre_gpu['USEDTIME']<'49days'], col='PARTITION', hue='PARTITION')
g.map_dataframe(sns.scatterplot, 
      x='GPUS', 
      y="USEDTIME_s");

Curious about jobs that ran as long or longer than requested time

In [ ]:
accre_gpu[accre_gpu['USEDTIME'] >= accre_gpu['REQTIME']]

In [ ]:
accre_gpu[accre_gpu['USEDTIME'] >= accre_gpu['REQTIME']].describe()

#### 4. What is the distribution of different groups and users accessing each partition? In each partition, who are the top users, and do they represent a majority of the runtime-weighted jobs on the partition?

In [ ]:
#number of accounts accessing each partition
(
    accre_gpu
    .groupby('PARTITION')['ACCOUNT']
    .nunique()
    .to_frame()
    .reset_index()
)

In [ ]:
#looking to see if users are associated with more than one account
#waiting to hear back if these actually are the same individuals, or if every account/user combo is unique
accre_gpu.groupby('USER')['ACCOUNT'].nunique().sort_values(ascending=False).head(25)

In [ ]:
#number of users accessing each partition (assuming here each username is unique regardless of account)
(
    accre_gpu
    .groupby('PARTITION')['USER']
    .nunique()
    .to_frame()
    .reset_index()
)

In [ ]:
#number of jobs by partition and STATE
(
    accre_gpu
    .groupby(['PARTITION', 'STATE'])['JOBID']
    .nunique()
    .to_frame()
    .reset_index()
)

In [ ]:
#total usage time by partition
(
    accre_gpu
    .groupby('PARTITION')['USEDTIME']
    .sum()
    .to_frame()
    .reset_index()
)

In [ ]:
#total memory use by partition
(
    accre_gpu
    .groupby('PARTITION')['USEDMEM']
    .sum()
    .to_frame()
    .reset_index()
)

In [ ]:
#plotting number of users by partition and account
sns.set(rc={"figure.figsize":(6, 8)})
sns.barplot(
             x='USER',
             y='ACCOUNT',
             hue='PARTITION',
             data=(accre_gpu
                    .groupby(['PARTITION', 'ACCOUNT'])['USER']
                    .nunique()
                    .to_frame()
                    .reset_index()
                    .sort_values('ACCOUNT'))
            );

In [ ]:
#plotting number of jobs by partition and account
sns.set(rc={"figure.figsize":(10, 8)})
sns.barplot(
             x='JOBID',
             y='ACCOUNT',
             hue='PARTITION',
             data=(accre_gpu
                    .groupby(['PARTITION', 'ACCOUNT'])['JOBID']
                    .nunique()
                    .to_frame()
                    .reset_index()
                    .sort_values('ACCOUNT'))
            );

In [ ]:
#plotting total memory usage by partition and account
sns.set(rc={"figure.figsize":(10, 8)})
sns.barplot(
             x='USEDMEM',
             y='ACCOUNT',
             hue='PARTITION',
             data=(accre_gpu
                    .groupby(['PARTITION', 'ACCOUNT'])['USEDMEM']
                    .sum()
                    .to_frame()
                    .reset_index()
                    .sort_values('ACCOUNT'))
            );

In [ ]:
#plotting total time used in minutes by partition and account
sns.set(rc={"figure.figsize":(10, 8)})
sns.barplot(
             x='USEDTIME_s',
             y='ACCOUNT',
             hue='PARTITION',
             data=(accre_gpu
                    .groupby(['PARTITION', 'ACCOUNT'])['USEDTIME_s']
                    .sum()
                    .to_frame()
                    .reset_index()
                    .sort_values('ACCOUNT'))
            );

In [ ]:
#plotting total time difference in m by partition and account
sns.set(rc={"figure.figsize":(10, 8)})
sns.barplot(
             x='time_dif_s',
             y='ACCOUNT',
             hue='PARTITION',
             data=(accre_gpu
                    .groupby(['PARTITION', 'ACCOUNT'])['time_dif_s']
                    .sum()
                    .to_frame()
                    .reset_index()
                    .sort_values('ACCOUNT'))
            );

In [ ]:
#number of jobs either completed, failed, or ran out of memory for each user
user_counts = (
    accre_gpu
    .groupby(['PARTITION','ACCOUNT','USER'])['JOBID']
    .count()
    .to_frame()
    .reset_index()
    .sort_values('JOBID', ascending=False)
    .rename(columns={'JOBID':'job_count'})
)
user_counts

Almost half of all jobs in this data set came frome one user in one partition

In [ ]:
#total amount of time used for all jobs either completed, failed, or ran out of memory for each user
user_time = (
    accre_gpu
    .groupby(['PARTITION', 'ACCOUNT', 'USER'])['USEDTIME']
    .sum()
    .to_frame()
    .reset_index()
)
user_time

In [ ]:
#total amount of memory used for all jobs either completed, failed, or ran out of memory for each user
user_memory = (
    accre_gpu
    .groupby(['PARTITION', 'ACCOUNT', 'USER'])['USEDMEM']
    .sum()
    .to_frame()
    .reset_index()
)
user_memory

In [ ]:
#total difference between requested and used time for each user
user_dif = (
    accre_gpu
    .groupby(['PARTITION', 'ACCOUNT', 'USER'])['time_dif']
    .sum()
    .to_frame()
    .reset_index()
)
user_dif

In [ ]:
#putting all these user details into a single df
from functools import reduce

dfs=[user_counts, user_memory, user_time, user_dif]

user_details = reduce(lambda left, right: pd.merge(left, right), dfs)

user_details['time_dif_h'] = user_details['time_dif'].astype('timedelta64[h]')

user_details['time_dif_h_per_job'] = user_details['time_dif'].astype('timedelta64[h]')/user_details['job_count']
user_details['mem_per_job'] = user_details['USEDMEM']/user_details['job_count']
user_details['time_per_job'] = user_details['USEDTIME']/user_details['job_count']

user_details['time_weighted_jobs'] = (user_details['USEDTIME'].dt.total_seconds()/86400) * (user_details['job_count'])

user_details

In [ ]:
user_details.describe()

#### 5. Currently there is a 5 day limit on runtime for GPU jobs, although some users have been asking for extensions. What is the distribution of requested runtime and actual runtime on jobs on each partition? Do users really need more time, or are they simply always requesting the maximum?

In [ ]:
#how many times did accounts make request for 5 or more days
accre_gpu.loc[accre_gpu['REQTIME'] >= '5 days']['ACCOUNT'].value_counts()

In [ ]:
#how many times did accounts have time differences between requested/used time of at least 1 day (for any amount of time requested)
accre_gpu.loc[accre_gpu['time_dif'] >= '1 day']['ACCOUNT'].value_counts()

In [ ]:
#looking at a plot of the count of different time differences (in hours) with each bin being 3 hours wide
sns.set(rc={"figure.figsize":(14, 10)})
sns.histplot(
    data = accre_gpu.loc[accre_gpu['time_dif_s'] >= 0], 
    x = (accre_gpu['time_dif'].astype('timedelta64[h]')),
    binwidth=3,
    hue = 'PARTITION'
);

In [ ]:
#looking more specifically at requests that are for 5 or more days, bins here are percentage of jobs rather than counts
print("Number of 5+ day requests: ")
print(len(accre_gpu.loc[accre_gpu['REQTIME'] >= '5 days']))
sns.set(rc={"figure.figsize":(14, 10)})
sns.histplot(
    data = accre_gpu.loc[accre_gpu['REQTIME'] >= '5 days'], 
    x = (accre_gpu['time_dif'].astype('timedelta64[h]')),
    stat='percent',
    binwidth=3,
    hue = 'PARTITION'
);

In [ ]:
#looking at the sum of all time differences by partition and account
sns.set(rc={"figure.figsize":(10, 8)})
sns.barplot(
             x='time_dif_h',
             y='ACCOUNT',
             hue='PARTITION',
             data=(user_details
                    .groupby(['PARTITION', 'ACCOUNT'])['time_dif_h']
                    .sum()
                    .to_frame()
                    .reset_index()
                    .sort_values('ACCOUNT'))
            );

In [ ]:
#looking at sum of time differences for those users who have instances of differences of at least 4 days
sns.set(rc={"figure.figsize":(10, 8)})
sns.barplot( x='time_dif_h',
             y='USER',
             hue='PARTITION',
             data=(user_details
                    .loc[user_details['USER']
                         .isin(accre_gpu.loc[accre_gpu['time_dif'] >= '4 days']['USER'])]
                    .groupby(['PARTITION', 'USER'])['time_dif_h']
                    .sum()
                    .to_frame()
                    .reset_index()
                    .sort_values('USER'))
            );